In [38]:
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sklearn
from sklearn.model_selection import train_test_split

# Data (not the newest data)

In [39]:
no_pieces = 9
X_list = [] # confidence_list
y_list =[]

for instance in range(1000):
    p_gold  = [0.0 for i in range(no_pieces-1)] + [1.0]
    random.shuffle(p_gold)

    noise_std = 0.1  
    p_noise = [np.abs(np.random.normal(scale = noise_std, loc = i)) for i in p_gold]
    smax = [i/np.sum(p_noise) for i in p_noise]

    X_list.append(smax)
    y_list.append(np.argmax(smax))
 
# Dataset 
X = torch.Tensor(X_list)
y = torch.LongTensor(y_list)
print (len(X), len(y))

#print (X, y)
X.shape

# Data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

1000 1000


# Hyperparameter

In [92]:
input_size = X.shape[1]
hidden_size = 7
output_size = 9
lr = 0.01
n_epochs = 30
report_every = 1

# Model

In [93]:
class Net(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        
        self.hidden = nn.Linear(input_size, hidden_size)
        self.output = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.hidden(x))
        x = F.relu(self.output(x))
        #x = torch.argmax(x, dim=1)

        return x

In [94]:
net = Net(input_size, hidden_size, output_size)

# Training

In [95]:
# Update the weights
# create optimizer
optimizer = optim.SGD(net.parameters(), lr=lr)

for epoch in range(n_epochs):
    total_loss = 0
    
    ### in training loop ###
    for instance in X_train:
        optimizer.zero_grad()
        # Loss Function
        pred = net(X_train)
        target = y_train
        criterion = nn.CrossEntropyLoss()
        loss = criterion(pred, target)
        total_loss += loss
        loss.backward()
        optimizer.step()
        
    if ((epoch+1) % report_every) == 0:
        print('epoch: %d, loss: %.4f' % (epoch, total_loss/len(X_train)))
        


epoch: 0, loss: 2.1385
epoch: 1, loss: 2.0366
epoch: 2, loss: 1.8816
epoch: 3, loss: 1.6620
epoch: 4, loss: 1.4100
epoch: 5, loss: 1.1800
epoch: 6, loss: 1.0010
epoch: 7, loss: 0.8696
epoch: 8, loss: 0.7725
epoch: 9, loss: 0.7001
epoch: 10, loss: 0.6462
epoch: 11, loss: 0.6068
epoch: 12, loss: 0.5777
epoch: 13, loss: 0.5559
epoch: 14, loss: 0.5392
epoch: 15, loss: 0.5260
epoch: 16, loss: 0.5155
epoch: 17, loss: 0.5070
epoch: 18, loss: 0.4999
epoch: 19, loss: 0.4940
epoch: 20, loss: 0.4890
epoch: 21, loss: 0.4847
epoch: 22, loss: 0.4811
epoch: 23, loss: 0.4779
epoch: 24, loss: 0.4751
epoch: 25, loss: 0.4726
epoch: 26, loss: 0.4704
epoch: 27, loss: 0.4684
epoch: 28, loss: 0.4666
epoch: 29, loss: 0.4649


# Testing

In [96]:
correct = 0

with torch.no_grad():
    for instance in y_test:
        target = y_test
        pred = net(X_test)
        pred_label = torch.argmax(pred, dim=1)
        
    print (pred_label)
    print (target)
        
    correct += torch.sum(pred_label==target).item()    
    print('test accuracy: %.2f' % (100.0 * correct / len(y_test)))

tensor([3, 2, 1, 2, 6, 0, 0, 4, 4, 7, 6, 6, 0, 2, 7, 8, 6, 0, 6, 2, 8, 0, 3, 3,
        4, 0, 4, 6, 2, 2, 4, 3, 0, 3, 2, 4, 0, 2, 2, 2, 1, 4, 3, 3, 3, 7, 0, 0,
        0, 4, 7, 0, 1, 1, 2, 2, 7, 0, 8, 7, 2, 2, 7, 8, 4, 8, 2, 0, 3, 2, 2, 1,
        4, 7, 8, 6, 3, 1, 1, 2, 8, 8, 8, 0, 4, 8, 6, 4, 1, 3, 3, 0, 6, 3, 4, 1,
        2, 3, 6, 3, 6, 4, 3, 1, 8, 4, 6, 0, 4, 4, 3, 0, 8, 3, 8, 6, 6, 2, 0, 4,
        8, 8, 0, 1, 2, 8, 3, 0, 4, 3, 3, 6, 6, 7, 8, 2, 8, 0, 8, 3, 2, 2, 1, 7,
        0, 0, 0, 7, 1, 1, 7, 8, 6, 4, 2, 3, 3, 1, 4, 6, 0, 8, 2, 0, 4, 0, 2, 6,
        8, 8, 8, 0, 3, 8, 1, 1, 7, 2, 4, 0, 1, 1, 2, 3, 2, 0, 0, 3, 7, 4, 3, 1,
        3, 6, 3, 3, 1, 3, 3, 3, 4, 4, 7, 8, 2, 0, 2, 7, 3, 3, 6, 3, 2, 1, 1, 0,
        3, 1, 6, 7, 3, 3, 3, 6, 2, 0, 8, 2, 8, 1, 6, 0, 8, 8, 7, 6, 1, 0, 8, 6,
        7, 1, 6, 3, 8, 2, 0, 8, 4, 0, 2, 3, 4, 1, 4, 0, 7, 1, 7, 1, 0, 0, 4, 8,
        0, 6, 2, 8, 1, 6, 7, 4, 0, 6, 6, 4, 0, 0, 0, 0, 4, 6, 6, 6, 2, 7, 3, 1,
        8, 6, 8, 8, 2, 1, 1, 0, 3, 8, 0,